<img src="../ancillarydata/logos/climbeco_course_logo.png" width="1000" align="left"/>
<a id='introduction'></a>
<br>

<br>
<br>


# <font color=#800000> Exercise 3</font>  
## _Upscaling eddy covariance GPP with remote sensing_
<br>

This exercise package is focusing on how to use remote sensing (RS) observations from satellite for upscaling eddy covariance measurements, based on the light use efficiency (LUE) model ``` GPP = PAR * fAPAR * LUE ```. ```PAR``` is the photosynthetically active radiation, ```fAPAR``` is the fractional absorbed PAR by vegetation, ```LUE``` is the efficiency of vegetation to convert the light energy into biomaterials. The aim of the exercise is to provide an insight into how remote sensing can be used to upscale GPP and to stimulate a discussion on strengths and weaknesses of this approach.


### Input Data:
- MODIS EVI2 observations from the Nadir Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) dataset ([MCD43A4 Version 6](https://lpdaac.usgs.gov/products/mcd43a4v006/)) during 2017-2018, aggregated to a 16-day temporal resolution to reduce noise. EVI2 is a vegetation index derived from red and near-infrared reflectance. fAPAR is estimated with the equation: ```fAPAR = (EVI2 - 0.1) * 1.25```, following [Zhang et al. (2017) Scientific Data, doi: 10.1038/sdata.2017.165](https://www.nature.com/articles/sdata2017165).


- 18 ICOS eddy-covariance (EC) sites with measurements during 2015-2018, including GPP and PAR. The measurements were aggregated to 16-day averages to match the MODIS data.


- MODIS land cover types data (a static map from [IGBP 2017](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1)).


- ECMWF ERA5 surface shortwave incoming radiation ([SSIR](https://www.ecmwf.int/)), resampled and aggregated to match the MODIS EVI2 data. PAR can be obtained by multiplying a scalar of 0.45 to SSIR: ```PAR = 0.45 * SSIR```.

<br>
<br>

The exercise includes the following parts:
1. [Calculate the LUE coefficients with EC and remote sensing data for different vegetation types](#part1).

2. [Create a LUE raster from the land cover classification map by assigning the calculated LUE values to certain land cover types](#part2).

3. [Upscale GPP from the remote sensing observations (e.g., EVI2), PAR estimations from ECMWF, and the created LUE raster](#part3).

4. [Investigate the upscaled GPP data](#part4).

5. [Tasks](#part5)

<br>
<br>

## Import modules
The code-cell below includes code for importing all the modules needed for completing the tasks.

In [ ]:
#Import modules:
import os
import pandas as pd
import numpy as np
import gdal
from sklearn import linear_model
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime
from IPython.display import display, HTML
from tqdm.notebook import tqdm

#Disable warnings:
import warnings
warnings.filterwarnings('ignore')

#Use Seaborn gray background grid:
sns.set(color_codes=True)

#Make single matplotlib plots being displayed at the center of the notebook:
display(HTML("""
<style>
.output_png img {
    display: block;
    margin-left: auto;
    margin-right: auto;
}
</style>
"""))

<br>

## Define paths
In the next code-cell we define the paths to the datafiles.

In [ ]:
personal_home = os.path.expanduser('~')
path_rs = personal_home + '/climbeco/data/rs/'
path_evi2 = path_rs +'EVI2/'
path_par = path_rs +'PAR/'

<div style="text-align: right"><a href="#introduction">Back to top</a></div>

<a id="part1"></a>

<br>
<br>

## <font color='#B22222'>Part 1</font> - Calculate the LUE coefficients for different vegetation types
In this part you will calculate the LUE coefficient for different vegetation types using EC measurements and satellite data. The data is found in _RS_GPP.xlsx_ and has been preprocessed to include both EC measurements info (e.g. timestamp, station name, vegetation type, GPP and PAR), MODIS data (EVI2) and fAPAR computed as ```fAPAR = (EVI2 - 0.1) * 1.25```.

<br>

### <font color='navy'>Import data</font>
Import excel-file containing ICOS flux data and processed MODIS data.


In [ ]:
#Read excel-file containing the following information:
#time, station, vegetation_type, GPP, PAR, MODevi2 & fAPAR
df = pd.read_excel(path_rs+'RS_GPP.xlsx',
                    converters = {'TIMESTAMP': pd.to_datetime})

#Sort dataframe by time:
df.sort_values(by = 'TIMESTAMP').head()

<br>

### <font color='navy'>Filter data</font>
GPP-values equal to 0 were filtered out to remove the influence from freezing temperature conditions. Also, PAR and  fAPAR values that were equal to NaN were also filtered out.

In [ ]:
#Filter out rows, where GPP = 0:
df_filtered = df.loc[(df.GPP>0) & (np.isnan(df.PAR)==False) & (np.isnan(df.fAPAR)==False)].copy()

#Show dataframe:
df_filtered.head()

<br>

### <font color='navy'>Add column with the product of _PAR_ and _fAPAR_ </font>
Add a new column to your Pandas DataFrame including the product of the columns _PAR_ and _fAPAR_.

In [ ]:
#Add a column containing the product of "PAR" and "fAPAR":
df_filtered['PAR_fAPAR_prod'] = df_filtered.PAR * df_filtered.fAPAR

#Show dataframe:
df_filtered.head()

<br>

### <font color='navy'>Set index</font>
Set the "TIMESTAMP" column as index and order index in ascending order.

In [ ]:
#Create a copy of your Pandas DataFrame & set "TIMESTAMP" as index:
df_ind = df_filtered.copy().set_index('TIMESTAMP').sort_index()

#Show dataframe:
df_ind.head()

<br>

### <font color='navy'>Overview</font>
You may get an overview over what the regression results might look like, by using the following piece of code:

```python
#Use Seaborn gray background grid:
sns.set(color_codes=True)

#Plot GPP/PAR*fAPAR regression:
sns.lmplot(x="PAR_fAPAR_prod",
           y="GPP",
           col='Type',
           data=df_filt_no_nans,
           col_wrap=3,
           col_order=sorted(list(df_filt_no_nans.Type.unique())));
```

<br>

which will generate:

<img src='../ancillarydata/images/exercise3/linear_regr.png'>

<br>

### <font color='navy'>Compute linear regression of GPP vs PAR * fAPAR per vegetation type</font>
You are given 3 ready functions:
- ```calc_Linear_Reg_GPP( )``` computes the linear regression of GPP and PAR * fAPAR for a given vegetation type
- ```plot_regr( )``` produces a plot of the linear regression results
- ```linear_reg_widget( )``` produces a form of widgets that allow the user to select a vegetation type and get the plot for the corresponding GPP vs PAR * fAPAR linear regression result.

Run the functions and use the widgets to extract the LUE values for every vegetation type.

In [ ]:
def calc_Linear_Reg_GPP(df, veg_type):
    
    '''
    Description: Function that computes the linear regression of GPP vs PAR * fAPAR
                 for a given vegetation type and returns the result.
    
    Input:       1. GPP & PAR*fAPAR (Pandas DataFrame)
                 2. Vegetation type (String)
             
    Output:      1. PAR*fAPAR   (Numpy Array)
                 2. GPP         (Numpy Array)
                 3. predictions (Numpy Array)
                 4. coefficient (Float)
                 5. intercept   (Float)
                 6. r2-score    (Float)
    
    '''
    
    #Define data/predictors:
    x = df.PAR_fAPAR_prod.loc[df.Type==veg_type]

    #Define target:
    target = pd.DataFrame(df.GPP.loc[df.Type==veg_type], columns=["GPP"])
    y = target.GPP
    
    #Reshape Pandas Series to a 2-dimensional array (rows, col)
    X = x.values.reshape(-1,1)
    Y = y.values.reshape(-1,1)
    
    #Select model:
    lm = linear_model.LinearRegression()

    #Fit model:
    model = lm.fit(X,Y)

    #Get predictions by the model for GPP:
    predictions = lm.predict(X)

    #Get coefficient/slope (here: LUE):
    coef = lm.coef_

    #Get intercept:
    intercept = lm.intercept_

    #Get R2-score:
    r2 = r2_score(Y, predictions)
    
    #Return values:
    return X, Y, predictions, round(coef[0][0], 4), round(intercept[0], 3), round(r2, 2)  

In [ ]:
def plot_regr(veg_type, x, y, predict, intercept, coef, r2):
    
    '''
    Description: Function that takes as input the output parameters of a
                 linear regression computation for a specific vegetation
                 type and creates a plot.
                
    Input:       1. Vegetation type (String)
                 2. PAR*fAPAR       (Numpy Array)
                 3. GPP             (Numpy Array)
                 4. predictions     (Numpy Array)
                 5. coefficient     (Float)
                 6. intercept       (Float)
                 7. r2-score        (Float)
    
    Output:      1. Matplotlib Plot
    
    '''
    
    #Use Seaborn gray background grid:
    sns.set(color_codes=True)
    
    #Create a plot (i.e. "figure") object and set the size of your plot:
    fig = plt.figure(figsize=(8, 5))

    #Plot outputs:
    plt.scatter(x, y, color='cornflowerblue')
    plt.plot(x, predict, color='firebrick', linewidth=3)

    #Add plot title:
    plt.title(veg_type)

    #Add axes labels:
    plt.xlabel('fAPAR * PAR')
    plt.ylabel('GPP')

    #Add annotation - regression formula:
    plt.text(x.min(), round(y.max())-1,
             'y = '+str(coef) +' x  + '+str(intercept),
             style='italic',
             bbox={'facecolor': '#FFED83', 
                   'alpha': 0.5,
                   'pad': 5})

    #Add annotation - R2:
    plt.text(x.min(), round(y.max())-3,
             'R\u00b2 = '+ str(r2),
             style='italic',
             bbox={'facecolor': '#f0c43d',
                   'alpha': 0.4,
                   'pad': 3})


    #Show plot:
    plt.show()
    

In [ ]:
def linear_reg_widget(df):
    
    '''
    Description: Function that creates a dropdown widget with different
                 vegetation types and a button. The user selects a
                 vegetation type from the dropdown list and once he/she
                 clicks on the "Update Plot" button, a new plot appears
                 with linear regression results for the selected
                 vegetation type.
                 
    Input:       GPP & PAR*fAPAR (Pandas DataFrame)
    
    Output:      Matplotlib Plot
    
    '''
    
    #Import modules:
    from ipywidgets import Dropdown, interact_manual
        
    #Create dropdown widget:
    veg_types = Dropdown(options=['Coniferous Forest',
                                  'Cropland',
                                  'Deciduous Forest',
                                  'Grassland',
                                  'Wetland'],
                         description='Vegetation:',
                         disabled=False)
    
    #Update plot:
    def update_func(vegetation):
        
        #Call function to get Linear-Regression results between "GPP/PAR*fAPAR" for "Coniferous Forest":
        x1, y1, predict, coef, interc, r2 = calc_Linear_Reg_GPP(df, vegetation)

        #Call function to plot Linear-Regression results between "GPP/PAR*fAPAR" for "Coniferous Forest:
        plot_regr(vegetation, x1, y1, predict, interc, coef, r2)
        
        
    #Create widget-form:    
    interact_c = interact_manual(update_func, vegetation=veg_types)

    #Format widgets:
    interact_c.widget.children[0].layout.width = '400px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 270px'
    interact_c.widget.children[1].description = 'Update Plot'
    interact_c.widget.children[1].button_style = 'danger'
    interact_c.widget.children[1].style.button_color = '#3973ac'
    interact_c.widget.children[1].layout.margin = '10px 10px 50px 440px' # top/right/bottom/left
    
    

In [ ]:
#Call function to display widgets:
linear_reg_widget(df_ind)

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<br>

<a id="part2"></a>

<br>

## <font color='#B22222'>Part 2</font> - Create a LUE raster
In this part, we will create a LUE raster. We first read in the MODIS land cover raster and plot it. Then the LUE raster is obtained by assigning the LUE values of different vegetation types (calculated above) to the MODIS land cover types (shown in the figure below). For example, Needleleaf forest types are assigned with a value of 0.096, croplands are assigned with a value of 0.0752, etc. The land cover code of IGBP can be found [here](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1). The raster has a spatial resolution of 500m. 

We also create a mask using the MODIS land cover raster. Land cover types that correspond to _background_ or _water bodies_ are assigned with _NaN_ while all other land cover types are assigned with _1_. 

<br>

### <font color='navy'>Read tiff with MODIS land cover</font>
Read the MODIS land cover tiff to a numpy array. The array stores land cover types as integers.

In [ ]:
#Open tiff-file with MODIS Landcover:
lc_tiff = gdal.Open(path_rs+'landcoverLC_sinprojection.tif')

#Import tiff as numpy array:
lc = lc_tiff.GetRasterBand(1).ReadAsArray()

#Convert values to int:
lc.astype(int)

#Set negative values to NaN:
lc[lc<0]=np.nan

<br>

### <font color='navy'>Get coordinates for the lower left and upper right corner of the raster</font>
In order to plot the MODIS land cover raster, we need to define its spatial extent. In other words, we need to know the values for its lower left and upper right corner. We can get this information using the gdal module's ```GetGeoTransform() ``` function. This function returns the minimum x-value, the raster resolution, the maximum y-value and the skewness coefficients of x and y. follow the [link](https://gdal.org/user/raster_data_model.html) for more information.

In [ ]:
#Get total num of columns in raster:
width = lc_tiff.RasterXSize

#Get total num of rows in raster:
height = lc_tiff.RasterYSize

# Get a list with:
# [min x-value, pixel width (≈500m), skewness coefficient of x,
#  max y-value, pixel height (≈500m), skewness coefficient of y]
gt = lc_tiff.GetGeoTransform() 

#Get coordinates for lower left corner and upper right corner of raster: 
min_x = gt[0]
min_y = gt[3] + width*gt[4] + height*gt[5] 
max_x = gt[0] + width*gt[1] + height*gt[2]
max_y = gt[3] 

<br>

### <font color='navy'>Plot MODIS land cover</font>
Create a plot of the MODIS land cover array using the combination of IGBP land cover code and color from [here](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1). 

In [ ]:
mpl.rcParams.update({'font.size': 11})
hfont = {'fontname':'Calibri'}


#Create figure to store plot:
fig = plt.figure(figsize = (11, 9)) 

#Add plot title:
plt.title('MODIS - Land Cover Map')

#Create colormap:
cmap = mpl.colors.ListedColormap(["#05450a", "#086a10",
                                  "#54a708", "#78d203", "#009900",
                                  "#c6b044", "#dcd159", "#dade48",
                                  "#fbff13", "#b6ff05", "#27ff87",
                                  "#c24f44", "#a5a5a5", "#ff6d4c",
                                  "#69fff8", "#f9ffa4", "#1c0dff"])

#Create plot by inserting the name of the raster/array as parameter:
m = plt.imshow(lc,        #raster/array to be plotted
               cmap=cmap, #colormap
               extent = (min_x, max_x, min_y+1, max_y)) #define spatial extent of raster to add coord-ticks

#Set location of colorbar ticks:
bounds = np.arange(1.5, 19, 0.95)

#Add colorbal label:
cbar = plt.colorbar(m, ticks=bounds)

#Set colorbar tick-labels:          
cbar.set_ticklabels(["Evergreen Needleleaf Forests",
                     "Evergreen Broadleaf Forests", 
                     "Deciduous Needleleaf Forests",
                     "Deciduous Broadleaf Forests",
                     "Mixed Forests",
                     "Closed Shrublands",
                     "Open Shrublands",
                     "Woody Savannas",
                     "Savannas",
                     "Grasslands",
                     "Permanent Wetlands",
                     "Croplands",
                     "Urban and Built-up Lands",
                     "Cropland/Natural Vegetation Mosaics",
                     "Permanent Snow and Ice",
                     "Barren",
                     "Water Bodies"])

# save plot as png-file
#plt.savefig('myplot.png')

#Show plot:
plt.show()

plt.close()

<br>

### <font color='navy'>Create LUE raster </font>
Create a copy of the MODIS land cover array. In the new array, assign the LUE values you calculated earlier for the different vegetation types to the different MODIS land cover types.

In [ ]:
#Create a copy of the MODIS landcover raster and convert its values to float:
LUE = lc.copy().astype(np.float32)

#Assign MODIS land cover types with the corresponding LUE-value:
LUE[LUE==1] = 0.096   #Evergreen Needleleaf Forests --> LUE for Coniferous Forest
LUE[LUE==2] = 0.096   #Evergreen Broadleaf Forests  --> LUE for Coniferous Forest
LUE[LUE==3] = 0.068   #Deciduous Needleleaf Forests --> LUE for Deciduous Forest
LUE[LUE==4] = 0.068   #Deciduous Broadleaf Forests  --> LUE for Deciduous Forest
LUE[LUE==5] = 0.068   #Mixed Forests                --> LUE for Deciduous Forest
LUE[LUE==6] = 0.068   #Closed Shrublands            --> LUE for Deciduous Forest
LUE[LUE==7] = 0.0583  #Open Shrublands              --> LUE for Grassland
LUE[LUE==8] = 0.0583  #Woody Savannas               --> LUE for Grassland
LUE[LUE==9] = 0.0583  #Savannas                     --> LUE for Grassland
LUE[LUE==10] = 0.0583 #Grasslands                   --> LUE for Grassland
LUE[LUE==11] = 0.0565 #Permanent Wetlands           --> LUE for Wetland
LUE[LUE==12] = 0.0752 #Croplands                    --> LUE for Cropland
LUE[LUE==13] = 0.0    #Urban & Built-up Lands
LUE[LUE==14] = 0.0752 #Cropland/Natural Veg. Mosaic --> LUE for Cropland
LUE[LUE==15] = 0.0    #Permanent Snow and Ice
LUE[LUE==16] = 0.0    #Barren
LUE[LUE==17] = 0.0    #Water Bodies

#Show the unique values of the array:
np.unique(LUE)

<br>

### <font color='navy'> Create mask excluding _background_ and _water bodies_ </font>
Create a mask from the MODIS land cover array to mask out cells with _background_ or _water bodies_ values. Cells with the aforementioned values will be assigned with _NaN_ whilst the rest of the cells will be assigned with _1_.

In [ ]:
#Mask out values that represent "Background" or "Water Bodies":
int_mask = np.where((lc !=0) & (lc != 17) & (lc>0), 1, np.nan)


<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<br>

<a id="part3"></a>

<br>

## <font color='#B22222'>Part 3</font> - GPP upscaling
In this part we will estimate GPP from MODIS EVI2 images, ERA5 SSIR data, and the LUE raster (calculated above) using the equation:<br> ```GPP = PAR * fAPAR * LUE```.

### <font color='navy'> Read MODIS EVI2 and ERA5 SSIR images (tiffs) to estimate GPP</font>
The MODIS EVI2 data covers the h18v03 tile. We have downloaded 16-day data for 2017 and 2018. The ERA5 SSIR data covers the same tile as MODIS EVI2. 
We have prepared a set of functions to help you calculate the upscaled GPP:
- ```read_tiff()``` reads a specified tiff-file from a specified folder and returns a NumPy array.
- ```apply_mask()```applies a mask over an array. Note that both the mask-array and the to-be-masked array have to have the same dimensions.
- ```calc_gpp()``` calculates the upscaled GPP using the equation: ``` GPP = (SSIR * 0.1 * 0.45)  * (EVI2 * 0.0001) - 0.1) * LUE ```.

The dimension of the MODIS EVI2 and SSIR arrays is: (2309, 2232).<br>
Longitudes: 2309<br>
Latitudes: 2232<br>
<br>
There are 46 MODIS EVI2 and SSIR files, of which 23 files include values for 2017 and 23 for 2018. The datasets have the same temporal resolution. Note that EVI2- and SSIR-data are stored as integers to reduce storage space. Therefore, in order to obtain the actual values, we need to multiple a scalar of 0.0001 to the EVI2 data and 0.1 to the radiation data. Also, keep in mind that ```PAR = SSIR * 0.45``` and ```fAPAR = (EVI2 - 0.1) * 1.25```.

Negative GPP values are assigned with 0. 

In [ ]:
#Function that reads a tiff into a numpy array:
def read_tiff(path, filename, mask=None):
    
    #Open file:
    tiff = gdal.Open(path + filename)
    
    if(mask is None):
    
        #Import tiff as numpy array:
        tiff_arr = tiff.GetRasterBand(1).ReadAsArray()
        
    else:
        
        #Import tiff as numpy array:
        tiff_arr = tiff.GetRasterBand(1).ReadAsArray()*mask
        
    
    #Return array:
    return tiff_arr    

In [ ]:
#Function that applies a mask over an array.
#Both input arrays must have the same dimensions.
def apply_mask(array, mask_array):
    
    return array * mask_array

In [ ]:
def calc_gpp(ssir, evi2, lue):
    
    #Compute upscaled gpp:
    gpp = ((ssir * 0.1 * 0.45) * ((evi2 * 0.0001) - 0.1) * 1.25 * lue)
    
    #Set negative gpp-values to zero:
    gpp[gpp<0] = 0
    
    #Return result:
    return gpp

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<br>

<a id="part4"></a>

<br>

## <font color='#B22222'>Part 4</font> - Investigate the upscaled GPP
Now that we have the functions to calculate upscaled GPP, we could compare the upcaled GPP between 2017 and 2018. Below are some examples of how to perform these comparisons. Go through the examples and elaborate on what other analysis methods could be used.

List of examples:

1. [Compare annual mean GPP of 2017 and 2018](#ex1)
2. [Compare mean GPP for summer (June-July) 2017 and summer (June-July) 2018](#ex2)
3. [Compare difference between 2017 & 2018 for EVI2 & PAR](#ex3)
4. [Statistics for different land cover types](#ex4)
5. [Compare EVI time series of different vegetation types](#ex5)
6. [Compare aggregated EVI time series for regions with different vegetation type](#ex6)

<a id="ex1"></a>

### <font color='navy'>Example 1: _Compare annual mean GPP of 2017 and 2018_ </font>
In this example, we will compute the annual mean of upscaled GPP for 2017 and 2018 and plot the difference.<br>
We will use the following functions:

- ```calc_annual_mean()``` returns the annual mean of upscaled GPP, EVI2 or PAR.
- ```plot_2D()``` creates a plot for a given 2D-array. You may pass the plot title, colorbar label and colorbar limits as input parameters.


In [ ]:
def calc_annual_mean(year, variable):
    
    #Check year:
    if year==2017:
        index=range(0,23) #indices for files with values for 2017
            
    elif year==2018:
        index=range(23,46) #indices of files with values for 2018
            
    else:
        print('Error!\nDataset only available for 2017-2018...')
    
    #Create & initialize array to store results:
    total_sum = np.zeros((LUE.shape[0], LUE.shape[1]))
    
    #If input-variable is "gpp":
    if(variable=='gpp'):
        
        #Get an ordered list of the available EVI2/PAR filenames:
        evi2_files = sorted(os.listdir(path_evi2))    
        par_files = sorted(os.listdir(path_par))
        
        #Loop through every file in the index:
        for i in tqdm(index):
            
            #Compute upscaled gpp and add it to total_sum:
            total_sum = np.add(total_sum, calc_gpp(read_tiff(path_par, par_files[i]), read_tiff(path_evi2, evi2_files[i]), LUE)) 
    
    
    #If input-variable is "evi2":    
    elif(variable=='evi2'):
        
        #Get an ordered list of the available EVI2 filenames:
        evi2_files = sorted(os.listdir(path_evi2)) 
        
        #Loop through every file in the index:
        for i in tqdm(index):
            
            #Read EVI2 tiff from current filename and add its corresponding array to the sum:
            total_sum = np.add(total_sum, read_tiff(path_evi2, evi2_files[i])*0.0001)
    
    
    #If input-variable is "par":  
    elif(variable=='par'):
        
        #Get an ordered list of the available PAR filenames:
        par_files = sorted(os.listdir(path_par))
        
        #Loop through every file in the index:
        for i in tqdm(index):
            
            #Read EVI2 tiff from current filename and add its corresponding array to the sum:
            total_sum = np.add(total_sum, read_tiff(path_par, par_files[i])*0.1*0.45)  
    
    
    #Return annual-mean array:
    return total_sum/len(index)   

In [ ]:
def plot_2D(array2D, plot_title, cbar_title, cbar_min_limit, cbar_max_limit):
    
    #Create figure to store plot:
    fig = plt.figure(figsize = (11, 9)) 
    plt.imshow(array2D,#[100:,:],
               cmap="Spectral_r",
               clim=(cbar_min_limit,cbar_max_limit),
               extent = (min_x, max_x, min_y, max_y)) 

    #Add colorbar:
    cbar = plt.colorbar()

    #Add colorbar title:
    cbar.set_label(cbar_title, rotation=270, labelpad=15)

    #Add title:
    plt.title(plot_title)

    #Show plot:
    plt.show()  

In [ ]:
#Calculate annual mean of upscaled GPP for 2017:
gpp_2017 = calc_annual_mean(2017, 'gpp')

In [ ]:
#Calculate annual mean of upscaled GPP for 2018:
gpp_2018 = calc_annual_mean(2018, 'gpp')

In [ ]:
#Calculate difference of GPP annual means:
gpp_annual_means_diff = gpp_2018 - gpp_2017

#Plot difference of GPP annual means:
plot_2D(apply_mask(gpp_annual_means_diff, int_mask), 'GPP annual mean difference (2018 - 2017)', 'GPP diff', -3, 3)

<div style="text-align: right"><a href="#part4">[Back to examples]</a></div>
<br>

<a id="ex2"></a>
<br>

### <font color='navy'>Example 2: _Compare mean GPP for summer (June-July) 2017 and summer (June-July) 2018_ </font>
This example shows how you can perform a similar comparison to example 1, by calculating the mean for just June-July 2017 and June-July 2018. Bear in mind that during the summer of 2018 the region covered by our dataset experienced a severe drought. Can you see which regions were most seriously affected by the drought in the map? Are there any stations near the affected areas and what is their dominant vegetation type?

In [ ]:
def calc_summer_mean(year, variable):
    
    #Check year:
    if year==2017:
        index=range(10,14) #indices for files with values for June-July 2017
            
    elif year==2018:
        index=range(33,37) #indices of files with values for June-July 2018
            
    else:
        print('Error!\nDataset only available for 2017-2018...')
        
    #Create & initialize array to store results:
    total_sum = np.zeros((LUE.shape[0], LUE.shape[1]))
    
    #Check variable:
    if(variable=='gpp'):
        
        #Get an ordered list of the available EVI2/PAR filenames:
        evi2_files = sorted(os.listdir(path_evi2))    
        par_files = sorted(os.listdir(path_par))
        
        #Loop through every file in the index:
        for i in tqdm(index):
            
            #Compute upscaled gpp and add it to total_sum:
            total_sum = np.add(total_sum, calc_gpp(read_tiff(path_par, par_files[i]), read_tiff(path_evi2, evi2_files[i]), LUE)) 
        
    #Return annual-mean array:
    return total_sum/len(index)   
        
    

In [ ]:
#Calculate summer mean GPP for 2017:
gpp_summer_mean_2017 = calc_summer_mean(2017, 'gpp')

In [ ]:
#Calculate summer mean GPP for 2018:
gpp_summer_mean_2018 = calc_summer_mean(2018, 'gpp')

In [ ]:
#Calculate difference of GPP annual summer means (2017-2018):
gpp_summer_means_diff = gpp_summer_mean_2018 - gpp_summer_mean_2017

#Plot difference of GPP summer annual means:
plot_2D(apply_mask(gpp_summer_means_diff, int_mask), 'GPP annual mean (June and July) difference (2018 - 2017)', 'GPP diff', -3, 3)

<div style="text-align: right"><a href="#part4">[Back to examples]</a></div>
<br>

<a id="ex3"></a>
<br>

### <font color='navy'>Example 3: _Compare difference between 2017 & 2018 for EVI2 & PAR_ </font>
Repeat the same process as described in examples 1 and 2 for EVI and PAR.

In [ ]:
#Extract EVI2 annual means:
evi2017 = calc_annual_mean(2017, 'evi2')
evi2018 = calc_annual_mean(2018, 'evi2')

#Calculate difference:
evi2_diff = evi2018 - evi2017

In [ ]:
#Extract PAR annual means:
par2017 = calc_annual_mean(2017, 'par')
par2018 = calc_annual_mean(2018, 'par')

#Calculate difference:
par_diff = par2018 - par2017

In [ ]:
#Plot results:
plot_2D(apply_mask(evi2_diff, int_mask), 'EVI2 annual mean difference (2018 - 2017)', 'EVI2 diff', None, None)
plot_2D(apply_mask(par_diff, int_mask), 'PAR annual mean difference (2018 - 2017)', 'PAR diff', None, None)

<div style="text-align: right"><a href="#part4">[Back to examples]</a></div>
<br>

<a id="ex4"></a>
<br>

### <font color='navy'> Example 4: Statistics for different land cover types </font>
In this example we will create a [boxplot](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51) of the difference between GPP annual mean of 2017 and 2018.

In [ ]:
#Get list of indices for every landcover type:
ls1 = np.where(lc == 1)
ls2 = np.where(lc == 2)
ls3 = np.where(lc == 3)
ls4 = np.where(lc == 4)
ls5 = np.where(lc == 5)
ls6 = np.where(lc == 6)
ls7 = np.where(lc == 7)
ls8 = np.where(lc == 8)
ls9 = np.where(lc == 9)
ls10 = np.where(lc == 10)
ls11 = np.where(lc == 11)
ls12 = np.where(lc == 12)
ls13 = np.where(lc == 13)
ls14 = np.where(lc == 14)
ls15 = np.where(lc == 15)
ls16 = np.where(lc == 16)

In [ ]:
#Get GPP mean diff between 2017 and 2018 for every landcover:
ls1_val = gpp_annual_means_diff[ls1]
ls2_val = gpp_annual_means_diff[ls2]
ls3_val = gpp_annual_means_diff[ls3]
ls4_val = gpp_annual_means_diff[ls4]
ls5_val = gpp_annual_means_diff[ls5]
ls6_val = gpp_annual_means_diff[ls6]
ls7_val = gpp_annual_means_diff[ls7]
ls8_val = gpp_annual_means_diff[ls8]
ls9_val = gpp_annual_means_diff[ls9]
ls10_val = gpp_annual_means_diff[ls10]
ls11_val = gpp_annual_means_diff[ls11]
ls12_val = gpp_annual_means_diff[ls12]
ls13_val = gpp_annual_means_diff[ls13]
ls14_val = gpp_annual_means_diff[ls14]
ls15_val = gpp_annual_means_diff[ls15]
ls16_val = gpp_annual_means_diff[ls16]

In [ ]:
#Create figure to store plot:
fig = plt.figure(figsize=(12,8))

#Create plot:
ax = fig.add_subplot(111)

#Add boxplot:
ax.boxplot([np.nan_to_num(ls1_val, nan=0),
            np.nan_to_num(ls2_val, nan=0),
            np.nan_to_num(ls3_val, nan=0),
            np.nan_to_num(ls4_val, nan=0),
            np.nan_to_num(ls5_val, nan=0),
            np.nan_to_num(ls6_val, nan=0),
            np.nan_to_num(ls7_val, nan=0),
            np.nan_to_num(ls8_val, nan=0),
            np.nan_to_num(ls9_val, nan=0),
            np.nan_to_num(ls10_val, nan=0),
            np.nan_to_num(ls11_val, nan=0),
            np.nan_to_num(ls12_val, nan=0),
            np.nan_to_num(ls13_val, nan=0),
            np.nan_to_num(ls14_val, nan=0),
            np.nan_to_num(ls15_val, nan=0),
            np.nan_to_num(ls16_val, nan=0)],
           labels=['Evergreen Needleleaf Forests',
                   'Evergreen Broadleaf Forests',
                   'Deciduous Needleleaf Forests',
                   'Deciduous Broadleaf Forests',
                   'Mixed Forests',
                   'Closed Shrublands',
                   'Open Shrublands',
                   'Woody Savannas',
                   'Savannas',
                   'Grasslands',
                   'Permanent Wetlands',
                   'Croplands',
                   'Urban and Built-up Lands',
                   'Cropland/Natural Vegetation Mosaics',
                   'Permanent Snow and Ice',
                   'Barren'],
           showfliers=False)

#Set title:
plt.title('Boxplot comparison for different land cover types')

#Rotate x-axis labels (i.e. land cover types):
plt.xticks(rotation=90)

#Add y-axis label:
plt.ylabel('GPP mean diff 2018-2017')

#Show plot:
plt.show()

#Close plot:
plt.close()

<div style="text-align: right"><a href="#part4">[Back to examples]</a></div>
<br>

<a id="ex5"></a>
<br>

### <font color='navy'>Example 5: Compare EVI time series for different vegetation types</font>
Use the Pandas DataFrame from Part 1 and extract the EVI2 values for a given vegetation type. Aggregate the EVI2 values from all stations having the same vegetation by averaging. Plot the result. Do stations with the same vegetation type have similar EVI2 time series? Try aggregating by station.

In [ ]:
#Get average daily values of EVI2 for cropland:
crop_df = df_ind.loc[df_ind.Type=='Cropland'].groupby('TIMESTAMP').mean()

#Get average daily values of EVI2 for coniferous forest:
conif_df = df_ind.loc[df_ind.Type=='Coniferous Forest'].groupby('TIMESTAMP').mean()

In [ ]:
#Create a plot (i.e. "figure") object and set the size of your plot:
fig, ax = plt.subplots(figsize=(16, 6))

#Plot EVI2 values for cropland:
ax.plot(crop_df.index.values, crop_df.MODevi2, label='Cropland')

#Plot EVI2 values for coniferous forest:
ax.plot(conif_df.index.values, conif_df.MODevi2, label='Coniferous Forest')

#Add plot title:
plt.title('16-day mean EVI2 per vegetation type (station cells - all stations)')

#Add x-axis label:
plt.xlabel('Time')

#Add y-axis label:
plt.ylabel('EVI2')

#Add legend:
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

#Show plot:
plt.show()

#Close plot:
plt.close()

<div style="text-align: right"><a href="#part4">[Back to examples]</a></div>
<br>

<a id='ex6'></a>

### <font color='navy'>Example 6: Compare aggregated EVI time series for regions with different vegetation type </font>
In example 5, you used the EVI2 values for EVI2 cells including the coordinates of a station. In this example, we will use all the cells of the EVI2 raster that correspond to a specific vegetation type and compute the mean of those cells for every date. This way we will produce a time series of aggregated EVI2 values per vegetation type that we can plot.

In [ ]:
def check_evi2(evi2_arr):
    
    #Multiply imported evi2 with scalar 0.0001 to get acCurate EVI2 values:
    evi2 = evi2_arr * 0.0001
    
    #Set EVI-values that are out of range [-1,1] to NaN:
    evi2[evi2<-1.0]=np.NaN
    
    #Return result:
    return evi2
  

In [ ]:
def time_series_per_veg_type(veg_type_indices, variable):
    
    if variable=='evi2':
        
        #Get an ordered list of the available EVI2 filenames:
        evi2_files = sorted(os.listdir(path_evi2)) 
        
        #Loop through every file in the index:
        #for i in tqdm(range(len(evi2_files))):
            
        #Get EVI2-values for cells representing a certain vegetation type.
        #Compute the mean for every timepoint
        #Return a list of tuples where every tuple represents:
        #(Timepoint, mean EVI2 from all cells defined as cropland in current file)
        time_series = pd.DataFrame([(modis_dates[i], np.nanmean(check_evi2(read_tiff(path_evi2, evi2_files[i])[veg_type_indices])))
                       for i in tqdm(range(len(evi2_files)))], columns=['Time', variable.upper()])
        
    #Return Pandas DataFrame with time series:    
    return time_series
   

In [ ]:
#Get a list of the dates for which MODIS-EVI2 is available from their corresponding filenames:
modis_dates = [datetime.strptime(modis_date[11:15] + ' ' +modis_date[15:18], '%Y %j')
               for modis_date in sorted(os.listdir(path_evi2))]

In [ ]:
#Get indices for cells representing coniferous forest using the land cover array:
conif = np.where((lc == 1) | (lc == 2))

#Get indices for cells representing croplands using the land cover array:
cropl = np.where((lc == 12) | (lc == 14))

In [ ]:
#Get EVI2 time series for vegetation type: "Coniferous Forest"
evi_conif_df = time_series_per_veg_type(conif, 'evi2')

#Get EVI2 time series for vegetation type: "Cropland"
evi_cropl_df = time_series_per_veg_type(cropl, 'evi2')

In [ ]:
#Import modules:
from matplotlib import pyplot as plt

#Create a plot (i.e. "figure") object and set the size of your plot:
fig, ax = plt.subplots(figsize=(16, 6))

#Plot values for Air Temperature (daily values):
ax.plot(evi_cropl_df.Time, evi_cropl_df.EVI2, label='Cropland')

#Plot values for daytime GPP (daily values):
ax.plot(evi_conif_df.Time, evi_conif_df.EVI2, label='Coniferous Forest')

#Add plot title:
plt.title('16-day mean EVI2 per vegetation type (all cells)')

#Add x-axis label:
plt.xlabel('Time')

#Add y-axis label:
plt.ylabel('EVI2')

#Add legend:
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

#Show plot:
plt.show()

#Close plot:
plt.close()

<a id='ex7'></a>

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<br>

<a id='part5'>

<br>

## <font color='#B22222'>Part 5</font> - Tasks
Based on the aforementioned parts, you are asked to complete the following tasks:
1. Does the estimated GPP capture the GPP reduction due to drought in 2018 as compared to 2017? What do you think are the possible reasons? How can we improve our estimates?


2. Does the LUE keep constant during the study period? If not, how can you improve the accuracy of LUE? What about calculting LUE for individual years?


3. Does the greeness index (EVI2) go down during the 2018 drought? How do EVI2 values of different vegetation types respond to the drought. E.g, compare forests with crops.


4. What factors may have an influence on the LUE values except from the different vegetation types? What about the influence of climate or nutrients? Can you think about methods to incrorporate these impacts?

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<br>

<br>
<br>
<br>
<br>
<img src="../ancillarydata/logos/climbeco_contributors_logo.png" width="1000" align="left"/>
<br>
<br>